In [1]:
import torch
import onnx
import onnxruntime as ort
import netron

In [2]:
print(torch.__version__)
print(onnx.__version__)
print(ort.get_device())
print(ort.__version__)

2.7.0+cu126
1.18.0
GPU
1.22.0


In [3]:
MODEL_PATH = "./result/resnet50_best.pt"
ONNX_PATH = "./onnx/resnet50.onnx"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)

cuda


In [4]:
model = torch.load(MODEL_PATH, map_location=DEVICE, weights_only=False)
model.to(DEVICE)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
# 더미 생성: (batch, 1, mel_bins, time)
dummy = torch.randn(1, 1, 64, 101, device=DEVICE)

In [6]:
# onnx 모델 변환
print("ONNX 변환중...")

torch.onnx.export(model, # PyTorch 모델
                dummy, # 입력 텐서 (튜플도 가능)
                ONNX_PATH, # 저장할 파일 경로
                export_params=True, # 모델 파라미터 포함
                opset_version=11, # onnx의 연산자 버전
                input_names=['input'], # 입력 이름
                output_names=['output'], # 출력 이름
                dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}} # 배치 크기를 동적으로 지정
)

print(f"ONNX 변환완료'{ONNX_PATH}'에 저장됨")

ONNX 변환중...
ONNX 변환완료'./onnx/resnet50.onnx'에 저장됨


In [7]:
model = onnx.load(ONNX_PATH)
print(onnx.helper.printable_graph(model.graph))

graph main_graph (
  %input[FLOAT, batch_sizex1x64x101]
) initializers (
  %fc.weight[FLOAT, 5x2048]
  %fc.bias[FLOAT, 5]
  %onnx::Conv_497[FLOAT, 64x1x7x7]
  %onnx::Conv_498[FLOAT, 64]
  %onnx::Conv_500[FLOAT, 64x64x1x1]
  %onnx::Conv_501[FLOAT, 64]
  %onnx::Conv_503[FLOAT, 64x64x3x3]
  %onnx::Conv_504[FLOAT, 64]
  %onnx::Conv_506[FLOAT, 256x64x1x1]
  %onnx::Conv_507[FLOAT, 256]
  %onnx::Conv_509[FLOAT, 256x64x1x1]
  %onnx::Conv_510[FLOAT, 256]
  %onnx::Conv_512[FLOAT, 64x256x1x1]
  %onnx::Conv_513[FLOAT, 64]
  %onnx::Conv_515[FLOAT, 64x64x3x3]
  %onnx::Conv_516[FLOAT, 64]
  %onnx::Conv_518[FLOAT, 256x64x1x1]
  %onnx::Conv_519[FLOAT, 256]
  %onnx::Conv_521[FLOAT, 64x256x1x1]
  %onnx::Conv_522[FLOAT, 64]
  %onnx::Conv_524[FLOAT, 64x64x3x3]
  %onnx::Conv_525[FLOAT, 64]
  %onnx::Conv_527[FLOAT, 256x64x1x1]
  %onnx::Conv_528[FLOAT, 256]
  %onnx::Conv_530[FLOAT, 128x256x1x1]
  %onnx::Conv_531[FLOAT, 128]
  %onnx::Conv_533[FLOAT, 128x128x3x3]
  %onnx::Conv_534[FLOAT, 128]
  %onnx::Conv_536[

In [8]:
try:
    onnx.checker.check_model(model)
    print("모델 검증 성공")
except onnx.checker.ValidationError as e:
    print("모델 검증 실패")
    print(e)

모델 검증 성공


In [9]:
# 모델 시각화
netron.start (ONNX_PATH)

Serving './onnx/resnet50.onnx' at http://localhost:8080


('localhost', 8080)